# Testing the Mediapipe handsign recognition

In [1]:
import mediapipe as mp
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time as t
from PIL import Image
from sklearn.model_selection import train_test_split
import csv
from IPython.display import clear_output

In [2]:
model = tf.keras.models.load_model("model1.h5")

In [3]:
signs = ["right", "left", "up", "down", "fist", "hand", "V"]
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.1, min_tracking_confidence=0.01)
drawingModule = mp.solutions.drawing_utils

def test(frame):
    results = hands.process(frame)
        
    if results.multi_hand_landmarks: 
        points = []
        for handLandmarks in results.multi_hand_landmarks:
            for landmark in handLandmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                points.append(x)
                points.append(y)
                points.append(z)
            drawingModule.draw_landmarks(frame, handLandmarks, mp_hands.HAND_CONNECTIONS)
        
        return True, np.array(points)
       
    else: 
        return False, 0

In [5]:
vc = cv2.VideoCapture(0)
cv2.namedWindow("preview")

while True:
    key = cv2.waitKey(20) & 0xFF
    rval, frame = vc.read()
    if rval == False:
        print(233)
    bol, res = test(frame)
    if bol:
        res = np.reshape(res, (1, res.shape[0]))
        res = np.argmax(model(res))
        cv2.putText(frame, signs[res] ,(250,80), 4,2, (0,0,0),2)
        cv2.imshow('preview', frame)
 
    elif key == ord(" "): 
        break

    else:
        cv2.imshow("preview", frame)

cv2.destroyAllWindows()
vc.release()
